# [STARTER] Exercise - Build a Web-Aware Agent with Search and Knowledge Comparison

In this exercise, you'll build an agent that can search the web for current information and compare
it with its internal knowledge. This demonstrates how to enhance an LLM's capabilities with real-time
web data and how to critically analyze differences between sources.


## Challenge

Your task is to create an agent that can:

- Implement web search functionality using Tavily API
- Parse and process search results effectively
- Handle different types of queries (news, facts, events)
- Extract relevant information from search results

## Setup
First, let's import the necessary libraries:

In [2]:
from pathlib import Path
import sys

nb_dir = Path(__file__).parent if "__file__" in globals() else Path.cwd()
parent = nb_dir.parent  # points to 03-building-agents
if str(parent) not in sys.path:
    sys.path.insert(0, str(parent))

In [3]:
import os
from datetime import datetime
from typing import List, Dict
from dotenv import load_dotenv
from tavily import TavilyClient

from lib.agents import Agent
from lib.messages import BaseMessage
from lib.tooling import tool

In [4]:
load_dotenv()

False

## Play with Tavily

In [5]:
api_key = os.getenv("TAVILY_API_KEY")
client = TavilyClient(api_key=api_key)

In [6]:
# [TODO] Define a query and run
query = "what is solana"
result = client.search(query)

In [7]:
result

{'query': 'what is solana',
 'follow_up_questions': None,
 'answer': None,
 'images': [],
 'results': [{'url': 'https://solana.com/learn/what-is-solana',
   'title': 'What is Solana?',
   'content': 'Solana is a high-performance network that enables fast, secure, and affordable digital transactions. It powers thousands of applications.',
   'score': 0.9396537,
   'raw_content': None},
  {'url': 'https://www.forbes.com/sites/digital-assets/article/what-is-solana-sol-how-it-works-and-what-to-know/',
   'title': 'What Is Solana (SOL)? How It Works And What To Know - Forbes',
   'content': 'Solana is a high-performance public blockchain platform that provides fast transaction speeds and low fees for developers and users, especially',
   'score': 0.92359936,
   'raw_content': None},
  {'url': 'https://www.coinbase.com/learn/crypto-basics/what-is-solana',
   'title': 'What is Solana (SOL)? - Coinbase',
   'content': 'Solana’s native cryptocurrency is [SOL](https://www.coinbase.com/price/sola

## Define Web Search tool

In [8]:
from lib.tools import web_search, compare_sources

In [9]:
tools = [web_search, compare_sources]

## Define Agents

The first agent should not use tools, just its own knowledge. The second one should have web search tool enabled.

In [10]:
# [TODO] Agent with no tools
simple_agent = Agent(
    model_name="gpt-4o-mini",
    instructions="You are a helpful assistant",
)

In [11]:
# [TODO] Agent with web search tool
instructions = (
    "You are a helpful assistant. Use web_search for time-sensitive or factual queries. "
    "Return 3–5 concise bullets with source titles and URLs; include a brief synthesis if available."
)

web_agent = Agent(model_name="gpt-4o-mini", instructions=instructions, tools=tools)


In [12]:
def print_messages(messages: List[BaseMessage]):
    for m in messages:
        print(f" -> (role = {m.role}, content = {m.content}, tool_calls = {getattr(m, 'tool_calls', None)})")

## Run your Agents

Run the Agents and compare them. The following queries are just for reference. Change them as you want.

**Simple Agent**

**Note**: This example relies on the date being recent enough that the answer will not be in the model's training data. Try with other current events/dates if needed to get similar results.

In [12]:
run1 = simple_agent.invoke(
    query="Who won the 2025 Oscar for International Movie?", 
)

print("\nMessages from run 1:")
messages = run1.get_final_state()["messages"]
print_messages(messages)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__

Messages from run 1:
 -> (role = system, content = You are a helpful assistant, tool_calls = None)
 -> (role = user, content = Who won the 2025 Oscar for International Movie?, tool_calls = None)
 -> (role = assistant, content = I'm sorry, but I don't have information about events or winners beyond October 2023, including the 2025 Oscars. You may want to check the latest sources or official announcements for the most current information., tool_calls = None)


In [13]:
print(run1.get_final_state()["messages"][-1].content)

I'm sorry, but I don't have information about events or winners beyond October 2023, including the 2025 Oscars. You may want to check the latest sources or official announcements for the most current information.


In [14]:
run2 = simple_agent.invoke(
    query="What are the most recent developments in AI technology?", 
)
print("\nMessages from run 2:")
messages = run2.get_final_state()["messages"]
print_messages(messages)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__

Messages from run 2:
 -> (role = system, content = You are a helpful assistant, tool_calls = None)
 -> (role = user, content = Who won the 2025 Oscar for International Movie?, tool_calls = None)
 -> (role = assistant, content = I'm sorry, but I don't have information about events or winners beyond October 2023, including the 2025 Oscars. You may want to check the latest sources or official announcements for the most current information., tool_calls = None)
 -> (role = user, content = What are the most recent developments in AI technology?, tool_calls = None)
 -> (role = assistant, content = As of my last knowledge update in October 2023, several significant developments in AI technology were making headlines:

1. **Generative AI Advancements**: Tools like OpenAI's ChatGPT and other generative models have become more sop

In [15]:
print(run2.get_final_state()["messages"][-1].content)

As of my last knowledge update in October 2023, several significant developments in AI technology were making headlines:

1. **Generative AI Advancements**: Tools like OpenAI's ChatGPT and other generative models have become more sophisticated, enabling better text generation, image synthesis, and even video creation. These models are being integrated into various applications, enhancing productivity tools and creative software.

2. **AI in Healthcare**: AI continues to make strides in healthcare, with advancements in diagnostic tools, personalized medicine, and drug discovery. Machine learning algorithms are increasingly used to analyze medical images and predict patient outcomes.

3. **AI Ethics and Regulation**: There has been a growing focus on the ethical implications of AI, leading to discussions about regulations to ensure responsible AI development and deployment. Governments and organizations are working on frameworks to address bias, privacy, and accountability in AI systems.

**Web Agent**

In [13]:
run1 = web_agent.invoke(
    query="Who won the 2025 Oscar for International Movie?", 
)

print("\nMessages from run 1:")
messages = run1.get_final_state()["messages"]
print_messages(messages)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__

Messages from run 1:
 -> (role = system, content = You are a helpful assistant. Use web_search for time-sensitive or factual queries. Return 3–5 concise bullets with source titles and URLs; include a brief synthesis if available., tool_calls = None)
 -> (role = user, content = Who won the 2025 Oscar for International Movie?, tool_calls = None)
 -> (role = assistant, content = None, tool_calls = [ChatCompletionMessageToolCall(id='call_FMPaJO6uWQxkkG5Bfm7MGAC8', function=Function(arguments='{"query":"2025 Oscar for International Movie winner","search_depth":"basic","max_results":5}', name='web_search'), type='function')])
 -> (role = tool, content = "{'answer': 'Brazil\\'s \"I\\'m Still Here\" won the 2025 Oscar for Best Internationa

In [14]:
print(run1.get_final_state()["messages"][-1].content)

The winner of the 2025 Oscar for Best International Feature Film was Brazil's *I'm Still Here*. Here are some key details:

- **Film**: *I'm Still Here*
- **Country**: Brazil
- **Plot**: The film is based on a true story set in 1970s Rio de Janeiro during a military dictatorship.
- **Notable Performance**: Fernanda Torres was nominated for her role as Eunice Paiva, and her mother, Fernanda Montenegro, also features in the film.

For further details, you can visit the following sources:
- [Oscars 2025 - Official Site](https://www.oscars.org/oscars/ceremonies/2025)
- [NPR Article on the Film](https://www.npr.org/2025/03/02/nx-s1-5315313/oscars-2025-im-still-here-brazil-best-international-feature)
- [Wikipedia - 97th Academy Awards](https://en.wikipedia.org/wiki/97th_Academy_Awards)


In [15]:
run2 = web_agent.invoke(
    query="What are the most recent developments in AI technology?", 
)
print("\nMessages from run 2:")
messages = run2.get_final_state()["messages"]
print_messages(messages)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__

Messages from run 2:
 -> (role = system, content = You are a helpful assistant. Use web_search for time-sensitive or factual queries. Return 3–5 concise bullets with source titles and URLs; include a brief synthesis if available., tool_calls = None)
 -> (role = user, content = Who won the 2025 Oscar for International Movie?, tool_calls = None)
 -> (role = assistant, content = None, tool_calls = [ChatCompletionMessageToolCall(id='call_FMPaJO6uWQxkkG5Bfm7MGAC8', function=Function(arguments='{"query":"2025 Oscar for International Movie winner","search_depth":"basic","max_results":5}', name='web_search'), type='function')])
 -> (role = tool, content = "{'answer': 'Brazil\\'s \"I\\'m Still Here\" won the 2025 Oscar for Best Internationa

In [16]:
print(run2.get_final_state()["messages"][-1].content)

Recent developments in AI technology as of 2023 include several notable trends and advancements:

- **Generative AI Growth**: There has been explosive growth in generative AI, which includes tools and applications that create content, such as text, images, and music.
- **Automated Machine Learning (AutoML)**: This trend allows organizations to automate the process of applying machine learning to real-world problems, making AI more accessible.
- **Natural Language Processing (NLP) Advancements**: Significant improvements have been made in NLP technologies, enabling better understanding and generation of human language.
- **Increased Business Adoption**: Businesses have experimented more with AI technologies, integrating them into various processes beyond consumer-facing applications.
- **AI Regulation and Literacy**: There is a growing focus on AI literacy and regulatory frameworks to address ethical and practical concerns related to AI technologies.

For more detailed information, you 

## Advanced

You can modify `agents.py` to include: 
- a comparison field in the state schema
- a web search step
- a comparison step in the workflow